<a target="_blank" href="https://colab.research.google.com/github/wakusoftware/curso-ml-espanol/blob/master/C1%20-%20Aprendizaje%20Supervisado/W2/C1_W2_Lab03_Escalado_Caracteristicas_Tasa_Aprendizaje.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup para Colab
Si estás corriendo este Notebook en Google Colab corre la celda de abajo, de lo contrario ignórala.

In [ ]:
!git clone https://github.com/wakusoftware/curso-ml-espanol.git

%cd curso-ml-espanol/C1 - Aprendizaje Supervisado/W2/

!cp lab_utils_common.py /content/

!cp lab_utils_multi.py /content/

!cp deeplearning.mplstyle /content/

!cp -r data /content/

!cp -r images /content/

%cd /content/

!rm -rf curso-ml-espanol/

# Lab: Escalado de características y tasa de aprendizaje (Multivariable)

## Objetivos
En este laboratorio vas a:
- Utilizar las rutinas de múltiples variables desarrolladas en el laboratorio anterior
- ejecutar el Descenso del Gradiente en un conjunto de datos con múltiples características
- explorar el impacto de la *tasa de aprendizaje alfa* en el descenso del gradiente
- mejorar el rendimiento del descenso del gradiente mediante el *escalado de características* usando la normalización z-score

## Herramientas
Utilizarás las funciones desarrolladas en el último laboratorio, así como matplotlib y NumPy.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lab_utils_multi import  load_house_data, run_gradient_descent 
from lab_utils_multi import  norm_plot, plt_equal_scale, plot_cost_i_w
from lab_utils_common import dlc
np.set_printoptions(precision=2)
plt.style.use('./deeplearning.mplstyle')


## Notación

|Notación General <br />  | Descripción| Python (si aplica) |
|: ------------|: ------------------------------------------------------------||
| $a$ | escalar, no en negrita                                                      ||
| $\mathbf{a}$ | vector, en negrita                                                 ||
| $\mathbf{A}$ | matriz, en negrita y mayúscula                                         ||
| **Regresión** |         |    |     |
|  $\mathbf{X}$ | matriz de ejemplos de entrenamiento                  | `X_train` |   
|  $\mathbf{y}$  | objetivos de los ejemplos de entrenamiento                | `y_train` 
|  $\mathbf{x}^{(i)}$, $y^{(i)}$ | Ejemplo de entrenamiento $i_{th}$ | `X[i]`, `y[i]`|
| m | número de ejemplos de entrenamiento | `m`|
| n | número de características en cada ejemplo | `n`|
|  $\mathbf{w}$  |  parámetro: peso                       | `w`    |
|  $b$           |  parámetro: sesgo                                           | `b`    |     
| $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ | El resultado de la evaluación del modelo en $\mathbf{x}^{(i)}$ parametrizado por $\mathbf{w},b$: $f_{\mathbf{w},b}(\mathbf{x}^{(i)}) = \mathbf{w} \cdot \mathbf{x}^{(i)}+b$  | `f_wb` | 
|$\frac{\partial J(\mathbf{w},b)}{\partial w_j}$| el gradiente o derivada parcial del costo con respecto a un parámetro $w_j$ |`dj_dw[j]`| 
|$\frac{\partial J(\mathbf{w},b)}{\partial b}$| el gradiente o derivada parcial del costo con respecto a un parámetro $b$| `dj_db`|

# Planteamiento del Problema

Como en los laboratorios anteriores, utilizarás el ejemplo motivador de la predicción de precios de viviendas. El conjunto de datos de entrenamiento contiene muchos ejemplos con 4 características (tamaño, habitaciones, pisos y edad) mostrados en la tabla a continuación. Nota, en este laboratorio, la característica de Tamaño está en pies cuadrados mientras que en laboratorios anteriores se utilizaban 1000 pies cuadrados. Este conjunto de datos es más grande que el del laboratorio anterior.

Nos gustaría construir un modelo de regresión lineal utilizando estos valores para que luego podamos predecir el precio de otras casas - digamos, una casa de 1200 pies cuadrados, 3 habitaciones, 1 piso, 40 años de antigüedad.

## Conjunto de Datos:
| Tamaño (pies cuadrados) | Número de Habitaciones  | Número de Pisos | Edad de la Vivienda | Precio (miles de dólares)  |   
| ----------------| ------------------- |----------------- |--------------|----------------------- |  
| 952             | 2                   | 1                | 65           | 271.5                  |  
| 1244            | 3                   | 2                | 64           | 232                    |  
| 1947            | 3                   | 2                | 17           | 509.8                  |  
| ...             | ...                 | ...              | ...          | ...                    |

In [ ]:
# cargar el conjunto de datos
X_train, y_train = load_house_data()
X_features = ['tamaño(pies cuadrados)','habitaciones','pisos','edad']

Vamos a visualizar el dataset y sus características graficando cada característica frente al precio.

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(12, 3), sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i], y_train)
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("Precio (miles)")
plt.show()


Graficar cada característica frente al objetivo, el precio, proporciona alguna indicación de qué características tienen la mayor influencia en el precio. Arriba, el aumento del tamaño también aumenta el precio. Las habitaciones y los pisos no parecen tener un impacto fuerte en el precio. Las casas más nuevas tienen precios más altos que las casas más antiguas.

<a name="toc_15456_5"></a>
## Descenso del Gradiente con Múltiples Variables
Aquí están las ecuaciones que desarrollaste en el último laboratorio sobre el descenso del gradiente para múltiples variables:

$$\begin{align*} \text{repetir}&\text{ hasta la convergencia:} \; \lbrace \newline\;
& w_j := w_j -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial w_j} \tag{1}  \; & \text{para j = 0..n-1}\newline
&b\ \ := b -  \alpha \frac{\partial J(\mathbf{w},b)}{\partial b}  \newline \rbrace
\end{align*}$$

donde, n es el número de características, los parámetros $w_j$,  $b$, se actualizan simultáneamente y donde  

$$
\begin{align}
\frac{\partial J(\mathbf{w},b)}{\partial w_j}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)})x_{j}^{(i)} \tag{2}  \\
\frac{\partial J(\mathbf{w},b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{\mathbf{w},b}(\mathbf{x}^{(i)}) - y^{(i)}) \tag{3}
\end{align}
$$
* m es el número de ejemplos de entrenamiento en el conjunto de datos

* $f_{\mathbf{w},b}(\mathbf{x}^{(i)})$ es la predicción del modelo, mientras que $y^{(i)}$ es el valor objetivo

## Tasa de Aprendizaje
<figure>
    <img src="./images/C1_W2_Lab06_learningrate.PNG" style="width:1200px;" >
</figure>
Las clases discutieron algunos de los problemas relacionados con la configuración de la tasa de aprendizaje $\alpha$. La tasa de aprendizaje controla el tamaño de la actualización de los parámetros. Vea la ecuación (1) arriba. Es compartida por todos los parámetros.

Vamos a ejecutar el descenso del gradiente y probar algunas configuraciones de $\alpha$ en nuestro conjunto de datos.

### $\alpha$ = 9.9e-7

In [ ]:
# establecer alpha a 9.9e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 9.9e-7)

Parece que la tasa de aprendizaje es demasiado alta. La solución no converge. El costo está *aumentando* en lugar de disminuir. Vamos a graficar el resultado:

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

El gráfico de la derecha muestra el valor de uno de los parámetros, $w_0$. En cada iteración, está sobrepasando el valor óptimo y, como resultado, el costo termina *aumentando* en lugar de acercarse al mínimo. Note que esta no es una imagen completamente precisa, ya que hay 4 parámetros que se modifican en cada paso y no solo uno. Este gráfico solo muestra $w_0$ con los otros parámetros fijados en valores benignos. En este y en gráficos posteriores, podrías notar que las líneas azul y naranja están ligeramente desplazadas.

### $\alpha$ = 9e-7
Probemos con un valor un poco menor y veamos qué sucede.

In [ ]:
# establecer alpha a 9e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 9e-7)


El costo está disminuyendo a lo largo de la ejecución, lo que muestra que el valor de alpha no es demasiado grande.

In [ ]:
plot_cost_i_w(X_train, y_train, hist)

A la izquierda, puedes ver que el costo está disminuyendo como debería. A la derecha, puedes ver que $w_0$ todavía oscila alrededor del mínimo, pero está disminuyendo en cada iteración en lugar de aumentar. Observa que `dj_dw[0]` cambia de signo en cada iteración ya que `w[0]` salta por encima del valor óptimo.
Este valor de alpha convergerá. Puedes variar el número de iteraciones para ver cómo se comporta.

### $\alpha$ = 1e-7
Probemos con un valor aún más pequeño para $\alpha$ y veamos qué sucede.

In [ ]:
# establecer alpha a 1e-7
_, _, hist = run_gradient_descent(X_train, y_train, 10, alpha = 1e-7)

El costo está disminuyendo a lo largo de la ejecución, lo que muestra que $\alpha$ no es demasiado grande.

In [ ]:
plot_cost_i_w(X_train,y_train,hist)

A la izquierda, puedes ver que el costo está disminuyendo como debería. A la derecha puedes ver que $w_0$ está disminuyendo sin cruzar el mínimo. Observa que `dj_w0` es negativo durante toda la ejecución. Esta solución también convergerá, aunque no tan rápidamente como en el ejemplo anterior.

## Escalado de Características
<figure>
    <img src="./images/C1_W2_Lab06_featurescalingheader.PNG" style="width:1200px;" >
</figure>
Las conferencias describieron la importancia de reescalar el conjunto de datos para que las características tengan un rango similar.
Si estás interesado en los detalles de por qué esto es así, haz clic en el encabezado 'detalles' a continuación. Si no, la sección siguiente te guiará a través de una implementación de cómo realizar el escalado de características.

<details>
<summary>
    <font size='3' color='darkgreen'><b>Detalles</b></font>
</summary>

Volvamos a mirar la situación con $\alpha$ = 9e-7. Este valor está bastante cerca del máximo que podemos establecer para $\alpha$ sin que diverja. Esta es una ejecución corta que muestra las primeras iteraciones:

<figure>
    <img src="./images/C1_W2_Lab06_ShortRun.PNG" style="width:1200px;" >
</figure>

Arriba, aunque el costo está disminuyendo, es claro que $w_0$ está progresando más rápidamente que los otros parámetros debido a su gradiente mucho mayor.

El gráfico a continuación muestra el resultado de una ejecución muy larga con $\alpha$ = 9e-7. Esto tarda varias horas.

<figure>
    <img src="./images/C1_W2_Lab06_LongRun.PNG" style="width:1200px;" >
</figure>
    
Arriba, puedes ver que el costo disminuyó lentamente después de su reducción inicial. Observa la diferencia entre `w0` y `w1`, `w2`, `w3`, así como `dj_dw0` y `dj_dw1-3`. `w0` alcanza su valor casi final muy rápidamente y `dj_dw0` ha disminuido rápidamente a un valor pequeño, mostrando que `w0` está cerca del valor final. Los otros parámetros se redujeron mucho más lentamente.

¿Por qué es esto? ¿Hay algo que podamos mejorar? Vea abajo:
<figure>
    <center> <img src="./images/C1_W2_Lab06_scale.PNG"   ></center>
</figure>   

La figura anterior muestra por qué las actualizaciones de $w$ son desiguales.
- $\alpha$ es compartido por todas las actualizaciones de parámetros ($w$ y $b$).
- el término de error común se multiplica por las características para los $w$ (no para $b$).
- las características varían significativamente en magnitud, lo que hace que algunas características se actualicen mucho más rápido que otras. En este caso, $w_0$ se multiplica por 'tamaño (pies cuadrados)', que generalmente es > 1000, mientras que $w_1$ se multiplica por 'número de habitaciones', que generalmente es 2-4.
    
La solución es el Escalado de Características.

En las clases se discutieron tres técnicas diferentes:
- Escalado de características, esencialmente dividiendo cada característica positiva por su valor máximo, o más generalmente, reescalar cada característica utilizando tanto su valor mínimo como máximo con la fórmula \((x - \text{min}) / (\text{max} - \text{min})\). Ambas formas normalizan las características al rango de -1 y 1, donde el primer método funciona para características positivas, es simple y sirve bien para el ejemplo de la conferencia, y el segundo método funciona para cualquier característica.
- Normalización por media: \(x_i := \dfrac{x_i - \mu_i}{\text{max} - \text{min}} \)
- Normalización Z-score, la cual exploraremos a continuación.

### Normalización Z-score
Después de la normalización Z-score, todas las características tendrán una media de 0 y una desviación estándar de 1.

Para implementar la normalización Z-score, ajusta tus valores de entrada como se muestra en esta fórmula:
$$x^{(i)}_j = \dfrac{x^{(i)}_j - \mu_j}{\sigma_j} \tag{4}$$ 
donde $j$ selecciona una característica o una columna en la matriz $\mathbf{X}$. $\mu_j$ es la media de todos los valores para la característica (j) y $\sigma_j$ es la desviación estándar de la característica (j).
$$
\begin{align}
\mu_j &= \frac{1}{m} \sum_{i=0}^{m-1} x^{(i)}_j \tag{5}\\
\sigma^2_j &= \frac{1}{m} \sum_{i=0}^{m-1} (x^{(i)}_j - \mu_j)^2  \tag{6}
\end{align}
$$

>**Nota de implementación:** Al normalizar las características, es importante
almacenar los valores utilizados para la normalización - el valor medio y la desviación estándar utilizados para los cálculos. Después de aprender los parámetros
del modelo, a menudo queremos predecir los precios de casas que no hemos
visto antes. Dado un nuevo valor de x (área de la sala y número de dormitorios), primero debemos normalizar x usando la media y la desviación estándar
que habíamos calculado previamente del conjunto de entrenamiento.

**Implementación**

In [ ]:
def zscore_normalize_features(X):
    """
    calcula X, normalizado por z-score por columna
    
    Args:
      X (ndarray (m,n))     : datos de entrada, m ejemplos, n características
      
    Retorna:
      X_norm (ndarray (m,n)): datos de entrada normalizados por columna
      mu (ndarray (n,))     : media de cada característica
      sigma (ndarray (n,))  : desviación estándar de cada característica
    """
    # encontrar la media de cada columna/característica
    mu     = np.mean(X, axis=0)                 # mu tendrá forma (n,)
    # encontrar la desviación estándar de cada columna/característica
    sigma  = np.std(X, axis=0)                  # sigma tendrá forma (n,)
    # elemento a elemento, resta mu para esa columna de cada ejemplo, divide por std para esa columna
    X_norm = (X - mu) / sigma      

    return (X_norm, mu, sigma)
 
# verificar nuestro trabajo
# from sklearn.preprocessing import scale
# scale(X_orig, axis=0, with_mean=True, with_std=True, copy=True)


Veamos los pasos involucrados en la normalización Z-score. El gráfico a continuación muestra el proceso de transformación paso a paso.

In [ ]:
mu = np.mean(X_train,axis=0)
sigma = np.std(X_train,axis=0)
X_mean = (X_train - mu)
X_norm = (X_train - mu)/sigma      

fig, ax = plt.subplots(1, 3, figsize=(12, 3))
ax[0].scatter(X_train[:,0], X_train[:,3])
ax[0].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[0].set_title("sin normalizar")
ax[0].axis('equal')

ax[1].scatter(X_mean[:,0], X_mean[:,3])
ax[1].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[1].set_title(r"X - $\mu$")
ax[1].axis('equal')

ax[2].scatter(X_norm[:,0], X_norm[:,3])
ax[2].set_xlabel(X_features[0]); ax[0].set_ylabel(X_features[3]);
ax[2].set_title(r"Normalizado Z-score")
ax[2].axis('equal')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
fig.suptitle("distribución de características antes, durante y después de la normalización")
plt.show()

El gráfico anterior muestra la relación entre dos de los parámetros del conjunto de entrenamiento, "edad" y "tamaño(sqft)". *Estos se representan con escala igual*.
- Izquierda: Sin normalizar: El rango de valores o la varianza de la característica 'tamaño(sqft)' es mucho más grande que la de 'edad'.
- Medio: El primer paso elimina la media o el valor promedio de cada característica. Esto deja características centradas alrededor de cero. Es difícil ver la diferencia para la característica 'edad', pero 'tamaño(sqft)' está claramente alrededor de cero.
- Derecha: El segundo paso divide por la varianza. Esto deja ambas características centradas en cero con una escala similar.

Normalicemos los datos y comparemos con los datos originales.

In [ ]:
# Normalizar las características originales
X_norm, X_mu, X_sigma = zscore_normalize_features(X_train)
print(f"X_mu = {X_mu}, \nX_sigma = {X_sigma}")
print(f"Rango pico a pico por columna en X sin procesar: {np.ptp(X_train,axis=0)}")   
print(f"Rango pico a pico por columna en X normalizado: {np.ptp(X_norm,axis=0)}")

El rango pico a pico de cada columna se reduce de un factor de miles a un factor de 2-3 mediante la normalización.

In [ ]:
fig, ax = plt.subplots(1, 4, figsize=(12, 3))
for i in range(len(ax)):
    norm_plot(ax[i], X_train[:, i])
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("conteo")
fig.suptitle("distribución de características antes de la normalización")
plt.show()
fig, ax = plt.subplots(1, 4, figsize=(12, 3))
for i in range(len(ax)):
    norm_plot(ax[i], X_norm[:, i])
    ax[i].set_xlabel(X_features[i])
ax[0].set_ylabel("conteo")
fig.suptitle("distribución de características después de la normalización")
plt.show()

Observa que arriba, el rango de los datos normalizados (eje x) está centrado alrededor de cero y aproximadamente entre +/- 2. Lo más importante es que el rango es similar para cada característica.

Volvamos a ejecutar nuestro algoritmo de descenso de gradiente con datos normalizados.
Nota el **valor mucho más grande de alpha**. Esto acelerará el descenso de gradiente.

In [ ]:
w_norm, b_norm, hist = run_gradient_descent(X_norm, y_train, 1000, 1.0e-1, )

Las características escaladas obtienen resultados **mucho, mucho más rápido** y con gran precisión. Observa que el gradiente de cada parámetro es muy pequeño al final de esta ejecución relativamente corta. Una tasa de aprendizaje de 0.1 es un buen punto de partida para la regresión con características normalizadas.
Ahora grafiquemos nuestras predicciones frente a los valores objetivo. Ten en cuenta que la predicción se realiza utilizando la característica normalizada, mientras que el gráfico se muestra utilizando los valores de características originales.

In [ ]:
# Predice el objetivo utilizando características normalizadas
m = X_norm.shape[0]
yp = np.zeros(m)
for i in range(m):
    yp[i] = np.dot(X_norm[i], w_norm) + b_norm

# Grafica las predicciones y los objetivos frente a las características originales    
fig,ax=plt.subplots(1,4,figsize=(12, 3),sharey=True)
for i in range(len(ax)):
    ax[i].scatter(X_train[:,i],y_train, label = 'objetivo')
    ax[i].set_xlabel(X_features[i])
    ax[i].scatter(X_train[:,i],yp,color=dlc["dlorange"], label = 'predicción')
ax[0].set_ylabel("Precio"); ax[0].legend();
fig.suptitle("objetivo versus predicción usando un modelo normalizado por puntuación z")
plt.show()

Los resultados lucen bien. Algunos puntos a tener en cuenta:
- Con múltiples características, ya no podemos tener un solo gráfico que muestre los resultados versus características.
- Al generar el gráfico, se utilizaron las características normalizadas. Cualquier predicción utilizando los parámetros aprendidos de un conjunto de entrenamiento normalizado también debe estar normalizada.

**Predicción**
El propósito de generar nuestro modelo es usarlo para predecir los precios de viviendas que no están en el conjunto de datos. Vamos a predecir el precio de una casa con 1200 pies cuadrados, 3 dormitorios, 1 piso, y 40 años de antigüedad. Recuerda que debes normalizar los datos con la media y la desviación estándar obtenidas cuando se normalizó el conjunto de datos de entrenamiento.

In [ ]:
# Primero, normalizamos nuestro ejemplo.
x_casa = np.array([1200, 3, 1, 40])
x_casa_norm = (x_casa - X_mu) / X_sigma
print(x_casa_norm)
x_prediccion_casa = np.dot(x_casa_norm, w_norm) + b_norm
print(f" precio predicho de una casa con 1200 pies cuadrados, 3 dormitorios, 1 piso, 40 años de antigüedad = ${x_prediccion_casa*1000:0.0f}")

**Contornos de Costo**  
<img align="left" src="./images/C1_W2_Lab06_contours.PNG"   style="width:240px;" >Otra forma de ver el escalado de características es en términos de los contornos de costo. Cuando las escalas de características no coinciden, la gráfica del costo versus parámetros en un gráfico de contorno es asimétrica.

En la gráfica a continuación, se ajusta la escala de los parámetros. La gráfica de la izquierda es el gráfico de contorno de costo de w[0], los pies cuadrados versus w[1], el número de dormitorios antes de normalizar las características. La gráfica es tan asimétrica que las curvas que completan los contornos no son visibles. En contraste, cuando las características están normalizadas, el contorno de costo es mucho más simétrico. El resultado es que las actualizaciones a los parámetros durante el descenso del gradiente pueden avanzar de manera igual para cada parámetro.

In [ ]:
plt_equal_scale(X_train, X_norm, y_train)

## ¡Felicidades!
En este laboratorio:
- utilizaste los procedimientos para regresión lineal con múltiples características que desarrollaste en laboratorios anteriores.
- exploraste el impacto de la tasa de aprendizaje $\alpha$ en la convergencia.
- descubriste el valor del escalado de características mediante la normalización de puntuaciones z para acelerar la convergencia.

## Reconocimientos
Los datos de viviendas se derivaron del conjunto de datos de viviendas de [Ames](http://jse.amstat.org/v19n3/decock.pdf) compilado por Dean De Cock para su uso en educación en ciencia de datos.